<a href="https://colab.research.google.com/github/tunyu0/two/blob/master/%E6%8E%A8%E8%AB%96%E7%84%A1%E4%BA%BA%E6%A9%9F_35_66.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sat Dec  3 10:21:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Download YOLOv7 repository and install requirements

%cd /content
!git clone https://github.com/augmentedstartups/yolov7.git
%cd yolov7
!pip install -r requirements.txt

/content
Cloning into 'yolov7'...
remote: Enumerating objects: 597, done.
remote: Total 597 (delta 0), reused 0 (delta 0), pack-reused 597
Receiving objects: 100% (597/597), 52.08 MiB | 17.22 MiB/s, done.
Resolving deltas: 100% (278/278), done.
/content/yolov7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 9.8 MB/s 


In [ ]:
import os
%cd /content/yolov7

!pip install -q --upgrade gdown
!gdown --id '1pNIakc_ojQ5txFTWYwXV-rrnTOkc69sV'
!unzip -q facemask.zip
os.remove("facemask.zip")

/content/yolov7
/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1pNIakc_ojQ5txFTWYwXV-rrnTOkc69sV
To: /content/yolov7/facemask.zip
100% 4.43G/4.43G [00:29<00:00, 148MB/s]


In [ ]:
yaml_file = "/content/yolov7/facemask/data.yaml"
if os.path.exists(yaml_file) == False:
  f= open(yaml_file,"w+")
  f.write("train: /content/yolov7/facemask/train/images\n") # train資料位置
  f.write("val: /content/yolov7/facemask/val/images\n")
  f.write("nc: 4\n") # 修改:有幾個類別
  f.write("\n")
  # 自己dataset的類別
  f.write("names: ['car', 'hov', 'person', 'motorcycle']\n") # 修改
  f.close()


In [ ]:
import os
import sys
sys.path.append('/content/yolov7')

import argparse
import time
from pathlib import Path
import cv2
import torch
import numpy as np
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized, TracedModel


def letterbox(img, new_shape=(640, 640), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True, stride=32):
    # Resize and pad image while meeting stride-multiple constraints
    shape = img.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:  # only scale down, do not scale up (for better test mAP)
        r = min(r, 1.0)

    # Compute padding
    ratio = r, r  # width, height ratios
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding
    if auto:  # minimum rectangle
        dw, dh = np.mod(dw, stride), np.mod(dh, stride)  # wh padding
    elif scaleFill:  # stretch
        dw, dh = 0.0, 0.0
        new_unpad = (new_shape[1], new_shape[0])
        ratio = new_shape[1] / shape[1], new_shape[0] / shape[0]  # width, height ratios

    dw /= 2  # divide padding into 2 sides
    dh /= 2

    if shape[::-1] != new_unpad:  # resize
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return img, ratio, (dw, dh)

In [ ]:
!gdown --id '11fgxgwyz147KUlYiqLWG7C39bU-Ame0I' --output data.zip
!unzip -q data.zip
os.remove("data.zip")

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=11fgxgwyz147KUlYiqLWG7C39bU-Ame0I
To: /content/yolov7/data.zip
100% 151M/151M [00:00<00:00, 173MB/s]


In [ ]:
classes_to_filter = None  #You can give list of classes to filter by name, Be happy you don't have to put class number. ['train','person' ]


opt  = {
    # 修改:權重位置很重要 很重要 很重要，尤其要注意exp
    "weights": "500_w6_training.pt", # Path to weights file default weights are for nano model
    "yaml"   : "facemask/data.yaml",
    "img-size": 1280, # default image size
    "conf-thres": 0.35, # confidence threshold for inference.
    "iou-thres" : 0.66, # NMS IoU threshold for inference.
    "device" : '0',  # device to run our model i.e. 0 or 0,1,2,3 or cpu
    "classes" : classes_to_filter  # list of classes to filter or None

}

In [ ]:
import cv2
import csv

def myinfer(source_image_path): 
  
  with torch.no_grad():
    weights, imgsz = opt['weights'], opt['img-size']
    set_logging()
    device = select_device(opt['device'])
    half = device.type != 'cpu'
    model = attempt_load(weights, map_location=device)  # load FP32 model
    stride = int(model.stride.max())  # model stride
    imgsz = check_img_size(imgsz, s=stride)  # check img_size
    if half:
      model.half()

    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]
    if device.type != 'cpu':
      model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))

    img0 = cv2.imread(source_image_path)
    img = letterbox(img0, imgsz, stride=stride)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).to(device)
    img = img.half() if half else img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
      img = img.unsqueeze(0)

    # Inference
    t1 = time_synchronized()
    pred = model(img, augment= False)[0]

    # Apply NMS
    classes = None
    if opt['classes']:
      classes = []
      for class_name in opt['classes']:

        classes.append(opt['classes'].index(class_name))


    pred = non_max_suppression(pred, opt['conf-thres'], opt['iou-thres'], classes= classes, agnostic= False)
    t2 = time_synchronized()
    for i, det in enumerate(pred):
      s = ''
      s += '%gx%g ' % img.shape[2:]  # print string
      gn = torch.tensor(img0.shape)[[1, 0, 1, 0]]
      if len(det):
        det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()

        for c in det[:, -1].unique():
          n = (det[:, -1] == c).sum()  # detections per class
          s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string
      
        for *xyxy, conf, cls in reversed(det):

          label = f'{names[int(cls)]} {conf:.2f}'
          plot_one_box(xyxy, img0, label=label, color=colors[int(cls)], line_thickness=3)

  # 把推論圖片的結果用csv存下來
  

  name = source_image_path.split('/')[-1]
  img_name = name.split('.')[0]
  csv_name = '/content/output_csv/' + name.split('.')[0] + ".csv"

  with open(csv_name, 'w', newline='') as csvfile:

    # 以空白分隔欄位，建立 CSV 檔寫入器
    writer = csv.writer(csvfile, delimiter=',') # 逗號分隔


    for i in pred[0].tolist():
      # i[0]左上x, i[1]左上y, i[2]右下x, i[3]右下y, i[4]信心, i[5]類別id      
      cls_id = int(i[5]) # 類別id
      bbox_LTx = int(np.round(i[0],0)) # 左上x
      bbox_LTy = int(np.round(i[1],0)) # 左上y
      bbox_W = int(np.round((i[2]-i[0]), 0)) # bbox寬
      bbox_H = int(np.round((i[3]-i[1]), 0)) # bbox高

      writer.writerow([img_name, cls_id, bbox_LTx, bbox_LTy, bbox_W, bbox_H])

   

  output_path =  '/content/output_images/' + name
  cv2.imwrite(output_path, img0)
  
  

In [ ]:
# 上傳測試集 Private Testing

!gdown --id '1DyZpY2AqTzHOYNg2L5SfbIuHF5l70e1F' --output data.zip
!unzip -q data.zip
os.remove("data.zip")

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1DyZpY2AqTzHOYNg2L5SfbIuHF5l70e1F
To: /content/yolov7/data.zip
100% 1.78G/1.78G [00:09<00:00, 194MB/s]


In [ ]:
import time
since = time.time() # 計時

# test圖片所在的資料夾
thepath = "/content/yolov7/Private Testing Dataset_v2"

for image in os.listdir(thepath):
  #print(image)
  img_path = thepath + '/' + image
  myinfer(source_image_path=img_path)

time_elapsed = time.time() - since

Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers.

In [ ]:
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

Training complete in 11m 10s


In [ ]:
!tar zcvf /content/csv.tgz /content/output_csv

tar: Removing leading `/' from member names
/content/output_csv/
/content/output_csv/img1854.csv
/content/output_csv/img1544.csv
/content/output_csv/img1634.csv
/content/output_csv/img1799.csv
/content/output_csv/img1974.csv
/content/output_csv/img1521.csv
/content/output_csv/img1838.csv
/content/output_csv/img1518.csv
/content/output_csv/img1821.csv
/content/output_csv/img1549.csv
/content/output_csv/img1830.csv
/content/output_csv/img1623.csv
/content/output_csv/img1576.csv
/content/output_csv/img1713.csv
/content/output_csv/img1862.csv
/content/output_csv/img1533.csv
/content/output_csv/img1603.csv
/content/output_csv/img1579.csv
/content/output_csv/img1797.csv
/content/output_csv/img1709.csv
/content/output_csv/img1965.csv
/content/output_csv/img1600.csv
/content/output_csv/img1659.csv
/content/output_csv/img1626.csv
/content/output_csv/img1662.csv
/content/output_csv/img1785.csv
/content/output_csv/img1812.csv
/content/output_csv/img1781.csv
/content/output_csv/img1973.csv
/conten

In [ ]:
# 上傳測試集 Public Testing

!gdown --id '1w8_OXLm3UvxNTYlliIvoz258iedpxH5-' --output data.zip
!unzip -q data.zip
os.remove("data.zip")

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1w8_OXLm3UvxNTYlliIvoz258iedpxH5-
To: /content/yolov7/data.zip
100% 1.87G/1.87G [00:31<00:00, 60.2MB/s]


In [ ]:
import cv2
import csv

def myinfer2(source_image_path): 
  
  with torch.no_grad():
    weights, imgsz = opt['weights'], opt['img-size']
    set_logging()
    device = select_device(opt['device'])
    half = device.type != 'cpu'
    model = attempt_load(weights, map_location=device)  # load FP32 model
    stride = int(model.stride.max())  # model stride
    imgsz = check_img_size(imgsz, s=stride)  # check img_size
    if half:
      model.half()

    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]
    if device.type != 'cpu':
      model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))

    img0 = cv2.imread(source_image_path)
    img = letterbox(img0, imgsz, stride=stride)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).to(device)
    img = img.half() if half else img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
      img = img.unsqueeze(0)

    # Inference
    t1 = time_synchronized()
    pred = model(img, augment= False)[0]

    # Apply NMS
    classes = None
    if opt['classes']:
      classes = []
      for class_name in opt['classes']:

        classes.append(opt['classes'].index(class_name))


    pred = non_max_suppression(pred, opt['conf-thres'], opt['iou-thres'], classes= classes, agnostic= False)
    t2 = time_synchronized()
    for i, det in enumerate(pred):
      s = ''
      s += '%gx%g ' % img.shape[2:]  # print string
      gn = torch.tensor(img0.shape)[[1, 0, 1, 0]]
      if len(det):
        det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()

        for c in det[:, -1].unique():
          n = (det[:, -1] == c).sum()  # detections per class
          s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string
      
        for *xyxy, conf, cls in reversed(det):

          label = f'{names[int(cls)]} {conf:.2f}'
          plot_one_box(xyxy, img0, label=label, color=colors[int(cls)], line_thickness=3)

  # 把推論圖片的結果用csv存下來
  

  name = source_image_path.split('/')[-1]
  img_name = name.split('.')[0]
  csv_name = '/content/output_csv2/' + name.split('.')[0] + ".csv"

  with open(csv_name, 'w', newline='') as csvfile:

    # 以空白分隔欄位，建立 CSV 檔寫入器
    writer = csv.writer(csvfile, delimiter=',') # 逗號分隔


    for i in pred[0].tolist():
      # i[0]左上x, i[1]左上y, i[2]右下x, i[3]右下y, i[4]信心, i[5]類別id      
      cls_id = int(i[5]) # 類別id
      bbox_LTx = int(np.round(i[0],0)) # 左上x
      bbox_LTy = int(np.round(i[1],0)) # 左上y
      bbox_W = int(np.round((i[2]-i[0]), 0)) # bbox寬
      bbox_H = int(np.round((i[3]-i[1]), 0)) # bbox高

      writer.writerow([img_name, cls_id, bbox_LTx, bbox_LTy, bbox_W, bbox_H])

   

  output_path =  '/content/output_images2/' + name
  cv2.imwrite(output_path, img0)
  
  

In [ ]:
import time
since = time.time() # 計時

# test圖片所在的資料夾
thepath = "/content/yolov7/public"

for image in os.listdir(thepath):
  #print(image)
  img_path = thepath + '/' + image
  myinfer2(source_image_path=img_path)

time_elapsed = time.time() - since

Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers... 
IAuxDetect.fuse
Fusing layers.

In [ ]:
!tar zcvf /content/pubic_csv.tgz /content/output_csv2

tar: Removing leading `/' from member names
/content/output_csv2/
/content/output_csv2/img1154.csv
/content/output_csv2/img1197.csv
/content/output_csv2/img1487.csv
/content/output_csv2/img1339.csv
/content/output_csv2/img1414.csv
/content/output_csv2/img1074.csv
/content/output_csv2/img1428.csv
/content/output_csv2/img1155.csv
/content/output_csv2/img1315.csv
/content/output_csv2/img1104.csv
/content/output_csv2/img1466.csv
/content/output_csv2/img1369.csv
/content/output_csv2/img1441.csv
/content/output_csv2/img1459.csv
/content/output_csv2/img1295.csv
/content/output_csv2/img1407.csv
/content/output_csv2/img1012.csv
/content/output_csv2/img1248.csv
/content/output_csv2/img1034.csv
/content/output_csv2/img1046.csv
/content/output_csv2/img1025.csv
/content/output_csv2/img1274.csv
/content/output_csv2/img1187.csv
/content/output_csv2/img1013.csv
/content/output_csv2/img1095.csv
/content/output_csv2/img1120.csv
/content/output_csv2/img1476.csv
/content/output_csv2/img1399.csv
/content/o

In [ ]:
!tar zcvf /content/pubic_img.tgz /content/output_images2

tar: Removing leading `/' from member names
/content/output_images2/
/content/output_images2/img1049.png
/content/output_images2/img1150.png
/content/output_images2/img1226.png
/content/output_images2/img1468.png
/content/output_images2/img1084.png
/content/output_images2/img1406.png
/content/output_images2/img1073.png
/content/output_images2/img1203.png
/content/output_images2/img1439.png
/content/output_images2/img1173.png
/content/output_images2/img1063.png
/content/output_images2/img1484.png
/content/output_images2/img1482.png
/content/output_images2/img1497.png
/content/output_images2/img1168.png
/content/output_images2/img1478.png
/content/output_images2/img1399.png
/content/output_images2/img1045.png
/content/output_images2/img1451.png
/content/output_images2/img1351.png
/content/output_images2/img1290.png
/content/output_images2/img1189.png
/content/output_images2/img1323.png
/content/output_images2/img1255.png
/content/output_images2/img1422.png
/content/output_images2/img1457